# Mississippi Valley Forest Birds Reforestation

This indicator was used in the 2022 base blueprint. 

Created by Amy Keister, last run by Amy Keister on 1 June 2023. It took 10 minutes to run.

In [47]:
import os
import arcpy

In [48]:
import time
start = time.time()

In [49]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\MississippiAlluvialValleyForestBirds_R\Ref.gdb"

In [50]:
# define final indicator outputs
Out = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\MississippiAlluvialValleyForestBirds_R\MississippiAlluvialValleyForestBirds_Reforestation.tif"

In [51]:
# define sub-indicator outputs

In [52]:
# define rasters used for cell size, extent, and snapping
Rextent= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"

In [53]:
# define inputs
FBB= r"F:\GIS_DATA\DecisionSupportTools\LMVJV\FBBDSM_2011_v2\FBBDSM_2011_v2.tif"
# use protection model to get full model extent
FP= r"F:\GIS_DATA\DecisionSupportTools\LMVJV\FPDSM\MAV_FPDSM_FINAL\MAV_FPDSM_v7_noCE_final_2.img"

## Start Analysis

In [54]:
# Set the workspace where I want the output to go
arcpy.env.workspace = OutWorkspace

In [55]:
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\MississippiAlluvialValleyForestBirds_R\Ref.gdb


## bring in LMVJV Reforstation DSM for interior forest breeding birds of the MAV

In [10]:
# Bring in the LMVJV tif
arcpy.conversion.RasterToGeodatabase(FBB, OutWorkspace, '')

<Result ''>

In [11]:
# take the above raster times one because it is acting weird
out_raster = arcpy.sa.Times("FBBDSM_2011_v2", 1); out_raster.save("FBB")

In [12]:
# make extent from forest protection raster
out_raster = arcpy.sa.Con(FP, 0, 0, "Value > -1 "); out_raster.save("full0")

In [13]:
# add in zeros
out_raster = arcpy.sa.CellStatistics("full0;FBB", "MAXIMUM", "DATA", "SINGLE_BAND", 90, "AUTO_DETECT"); out_raster.save("indicator")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent, clip and export to SA extent

In [60]:
# clip to SE 2022 Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ExtractByMask("indicator", Rextent); out_raster.save("SEMask")

In [61]:
# export as .tif
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("SEMask", Out, '', None, "15", "NONE", "NONE", "4_BIT", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [62]:
# set code block for next step

# From the documentation: "The final output was classified into 10 categories of reforestation priority based on percent 
# area (<10% of total model area, ≥10% and <20%, ≥20% and <30%, etc.) and clipped to the Mississippi Alluvial Valley Bird 
# Conservation Region boundary" 


codeblock = """
def Reclass(value):
    if value == 10:
        return '10 = Reforestation most likely to contribute to forest breeding bird habitat needs (≥90th percentile)'
    elif value == 9:
        return '9 = Reforestation most likely to contribute to forest breeding bird habitat needs (80th to <90th percentile)' 
    elif value == 8:
        return '8 = Reforestation more likely to contribute to forest breeding bird habitat needs (70th to <80th percentile)'
    elif value == 7:
        return '7 = Reforestation less likely to contribute to forest breeding bird habitat needs (60th to <70th percentile)'
    elif value == 6:
        return '6 = Reforestation least likely to contribute to forest breeding bird habitat needs (50th to <60th percentile)'
    elif value == 5:
        return '5 = Reforestation least likely to contribute to forest breeding bird habitat needs (40th to <50th percentile)'
    elif value == 4:
        return '4 = Reforestation least likely to contribute to forest breeding bird habitat needs (30th to <40th percentile)'
    elif value == 3:
        return '3 = Reforestation least likely to contribute to forest breeding bird habitat needs (20th to <30th percentile)'
    elif value == 2:
        return '2 = Reforestation least likely to contribute to forest breeding bird habitat needs (10th to <20th percentile)'
    elif value == 1:
        return '1 = Reforestation least likely to contribute to forest breeding bird habitat needs (<10th percentile)'
    elif value == 0:
        return '0 = Not a priority for reforestation'
"""

In [63]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(Out, "descript", "Reclass(!Value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\MississippiAlluvialValleyForestBirds_R\\MississippiAlluvialValleyForestBirds_Reforestation.tif'>

In [64]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 10:
		return 255
	if Value == 9:
		return 255
	if Value == 8:
		return 76
	if Value == 7:
		return 0
	if Value == 6:
		return 0
	if Value == 5:
		return 193
	if Value == 4:
		return 112
	if Value == 3:
		return 24
	if Value == 2:
		return 156
	if Value == 1:
		return 0
	else:
		return 255
		
def Reclass2(Value):
	if Value == 10:
		return 0
	if Value == 9:
		return 255
	if Value == 8:
		return 230
	if Value == 7:
		return 255
	if Value == 6:
		return 77
	if Value == 5:
		return 145
	if Value == 4:
		return 0
	if Value == 3:
		return 0
	if Value == 2:
		return 156
	if Value == 1:
		return 0
	else:
		return 255
		
def Reclass3(Value):
	if Value == 10:
		return 0
	if Value == 9:
		return 0
	if Value == 8:
		return 0
	if Value == 7:
		return 197
	if Value == 6:
		return 168
	if Value == 5:
		return 255
	if Value == 4:
		return 191
	if Value == 3:
		return 237
	if Value == 2:
		return 156
	if Value == 1:
		return 0
	else:
		return 255
		
"""

In [65]:
# calculate Red field
arcpy.management.CalculateField(Out, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(Out, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(Out, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\MississippiAlluvialValleyForestBirds_R\\MississippiAlluvialValleyForestBirds_Reforestation.tif'>

In [20]:
end = time.time()
print(end - start)

289.3651657104492
